## December 10th GLM From beginning to end with descriptions.

# Import it

In [7]:
import numpy as np
import pandas as pd
%matplotlib inline
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px
import warnings
warnings.filterwarnings('ignore')
from pathlib import Path
import statsmodels.api as sm
from fuzzywuzzy import fuzz
import statsmodels.api as sm
from statsmodels.stats.outliers_influence import variance_inflation_factor
import statsmodels.api as sm
import random
from imblearn.over_sampling import RandomOverSampler

from sklearn.model_selection import cross_validate
from sklearn.linear_model import LogisticRegression
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, balanced_accuracy_score, matthews_corrcoef


from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import precision_score, recall_score, f1_score, roc_auc_score, average_precision_score, balanced_accuracy_score, matthews_corrcoef

# Functions I wrote to do this 

In [8]:
def binarize_cols(df, nan_threshold=20):
    few_NaNs_cols = []
    for col in df.columns:
        if (df[col].nunique(dropna=False) == 2) or (df[col].nunique(dropna=True) == 2):
            unique_vals = df[col].unique()
            if (unique_vals[0] !=0 and unique_vals[1] !=0):
                print(f"col:{col}\n0 means:{unique_vals[0]}\n1 means: {unique_vals[1]}\n")
                df[col] = df[col].replace({unique_vals[0]: int(0), unique_vals[1]: int(1)})
            df[col].fillna(0, inplace=True) #now this will only make sense tho if we OHE first.    
    return df


def drop_columns(df, col_lst):
    for i in col_lst:
        if i in df.columns:
            df.drop(i, axis=1,inplace=True)
    return df

def lowercase_column(df, column_name):
    if column_name in df.columns:
        df[column_name] = df[column_name].astype(str).str.lower()
    return df

def count_candidates_by_district(df, district_column, new_column_name):
    district_counts = df.groupby(district_column).size().reset_index(name=new_column_name)
    df_merged = df.merge(district_counts, on=district_column)
    df_merged = df_merged[df_merged[new_column_name]>1]
    return df_merged

def move_column_to_front(df, col_lst):
    for column_name in col_lst:
        if column_name in df.columns:
            df = df[[column_name] + [col for col in df.columns if col != column_name]]
    return df

def get_info(df):
    print(f"COL VALUE TYPES \n{df.dtypes} \n\ndf shape:{df.shape}\n\nall the columns:\n{df.columns}")
    
    
def rename_column_if_exists(df, old_name, new_name):
    if old_name in df.columns:
        df.rename(columns={old_name: new_name}, inplace=True)
    return df
    
def replace_values_fill_na(df, column_name, replace_dict):
    if column_name in df.columns:
        df[column_name] = df[column_name].replace(replace_dict).fillna(0)
    return df
    
    
def get_ohe_cols(df, unique_limit=16, exclude = ['total_runners_house','total_runners']):
    #object_cols = df.select_dtypes(include=['object']).columns
    res = [col for col in df.columns if ((df[col].nunique(dropna=False) <= unique_limit) and (df[col].nunique(dropna=False)>2) and (col != 'total_runners') and (col != 'General Status') and (col !='Total Endorsements'))]
    return res


def convert_type(df, col_lst):
    for column_name in col_lst:
        if column_name in df.columns and df[column_name].dtype != 'int64':
            df[column_name] = df[column_name].astype(int)

            
def get_X_df(df, y_col):
    return df.drop(y_col, axis=1)

# def zero_dropper(df):
#     for col in df.columns:
#         if ((df[col]==0).mean() >= 0.90):
#             df = df.drop(col, axis=1)
#     return df

def drop_VIF_col(X, threshold=2):
    while True:
        vif_info = pd.DataFrame()
        vif_info['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
        vif_info['Column'] = X.columns
        vif_info = vif_info.sort_values('VIF', ascending=False)
        max_VIF = vif_info['VIF'].iloc[0]

        if max_VIF > threshold:
            to_drop = vif_info['Column'].iloc[0]
            print(f"Dropped: {to_drop} with VIF of {max_VIF}")
            X = X.drop(to_drop, axis=1)
        else:
            break

    return X  # Returning the modified DataFrame

def change_to_int(df, column):
    df[column] = df[column].astype(int)
    
    
def convert_to_int_if_possible(df, col_lst):
    for column_name in col_lst:
        temp_col = pd.to_numeric(df[column_name], errors='coerce')
        if temp_col.isna().sum() == 0:
            if all(temp_col.dropna() == temp_col.dropna().astype(int)):
                df[column_name] = temp_col.astype(int)
    return df

def aic(X, y):
    best_features = []
    best_aic = float('inf')

    for feature in X.columns:
        # Create a temporary DataFrame with the current set of best features plus the new feature
        X_temp = sm.add_constant(X[best_features + [feature]])

       
        model = sm.Logit(y, X_temp)  # Fit the logistic regression model
        result = model.fit()  # disp=0 suppresses the fit output

        # Check AIC and update if it is lower
        if result.aic < best_aic:
            best_aic = result.aic
            best_features.append(feature)

    # Fit the final model with the best features
    X_final = sm.add_constant(X[best_features])
    final_model = sm.Logit(y, X_final)
    final_result = final_model.fit()

    # You might want to return the final model, its summary, or AIC
    print(f"{best_features}")
    return final_result.summary(), final_result.aic



def zero_dropper(df):
    for col in df.columns:
        if ((df[col]==1).mean() >= 0.90):
            df = df.drop(col, axis=1)
    return df

In [9]:
FEC_merged = pd.read_csv('dec10.csv')
FEC_merged = convert_to_int_if_possible(FEC_merged,list(FEC_merged.columns))

## I'm going to OHE the columns that have more than two unique values (including NaNs) for example Emily Endorsed Yes, Emily Endorsed No, and Emily Endorsed NaNs.

In [10]:
ohe_cols =  get_ohe_cols(FEC_merged)

Now I want to OHE those columns and then drop one (to avoid multicollinearity). BEcasue one of our objectives is explainability and interpretabiliyt, I don't want to drop jsut any column ,I waot to drop the _NaN column or the "No information column" if it exists, and if not, drop the column with the most 1s. The function ``` ohe_and_avoid_multicollinearity``` does this.

In [45]:
def ohe_and_avoid_multicollinearity(df, columns_to_ohe):
    
    OHE_df = pd.get_dummies(df, columns = columns_to_ohe, drop_first=True)
    OHE_df = OHE_df[[col for col in OHE_df.columns if col not in columns_to_ohe]]
    return OHE_df

In [46]:
OHE_result = ohe_and_avoid_multicollinearity(FEC_merged, ohe_cols)

Now, I;m going to binarize columsn with exactly two unique values, and assing the result to binarized

In [47]:
binarized = binarize_cols(OHE_result)

Now, I'm going to drop any columns where a majority of the values are 1. 
This is called the variance threshold. 

from sklearn.feature_selection import VarianceThreshold

In [48]:
def low_variance_drop(df):
    for col in df.columns:
        if ((df[col]==1).mean() >= 0.80):
            df = df.drop(col, axis=1)
    return df

In [49]:
reduced_df = low_variance_drop(binarized)

Now, we do VIF

In [50]:
def drop_VIF_col(X, threshold=2):
    while True:
        vif_info = pd.DataFrame()
        vif_info['VIF'] = [variance_inflation_factor(X.values, i) for i in range(X.shape[1])]
        vif_info['Column'] = X.columns
        vif_info = vif_info.sort_values('VIF', ascending=False)
        max_VIF = vif_info['VIF'].iloc[0]

        if max_VIF > threshold:
            to_drop = vif_info['Column'].iloc[0]
            print(f"Dropped: {to_drop} with VIF of {max_VIF}")
            X = X.drop(to_drop, axis=1)
        else:
            break

    return X  # Returning the modified DataFrame


In [51]:
X = reduced_df.drop('General Status', axis=1)
best_X = drop_VIF_col(X)
best_X_lst = list(best_X.columns)

Dropped: Party Support?_1.0 with VIF of inf
Dropped: Position.on.Legalization.Decriminalization.of.Marijuana.Policy_Candidate provides no information with VIF of 354.87366495032614
Dropped: Position.on.Climate.Change_Candidate supports regulations and measures to combat climate change with VIF of 159.63060482195527
Dropped: Position.on.Gun.Control_Candidate supports gun control measures with VIF of 113.71850824525154
Dropped: Position.on.Campaign.Finance.Reform_Candidate provides no information with VIF of 77.07177348717686
Dropped: Total Endorsements with VIF of 68.53640281590212
Dropped: Position.on.Criminal.Justice.Reform_Candidate provides no information with VIF of 58.69578496907571
Dropped: Marital.Status_Married with VIF of 39.9321113020403
Dropped: Position.on.Federal.Taxes_Candidate supports raising taxes on the wealthy/corporations with VIF of 38.66106803497006
Dropped: Position.on.Social.Security_Candidate provides no information with VIF of 26.5974932101666
Dropped: Positio

In [55]:
best_X_lst

['Partisan Lean',
 'Race',
 'LGBTQ?',
 'Elected Official?',
 'Self-Funder?',
 'STEM?',
 'Listed.military.service.',
 'Obama Alum?_Yes',
 'Emily Endorsed?_1.0',
 'Gun Sense Candidate?_1.0',
 'Biden Endorsed?_1.0',
 'Warren Endorsed?_1.0',
 'Sanders Endorsed?_1.0',
 'Our Revolution Endorsed?_1.0',
 'Justice Dems Endorsed?_1.0',
 'PCCC Endorsed?_1.0',
 'Indivisible Endorsed?_1.0',
 'WFP Endorsed?_1.0',
 'VoteVets Endorsed?_1.0',
 'No Labels Support?_Yes',
 "Education_Bachelor's or some college",
 'Education_J.D.',
 "Education_Master's Degree (includes MBA)",
 'Education_Other Graduate',
 'Marital.Status_Engaged',
 'Marital.Status_Other',
 'Marital.Status_Single',
 'Marital.Status_Widowed',
 'Position.on.Affordable.Care.Act..ObamaCare._Candidate provides no information',
 'Position.on.Federal.Taxes_Candidate provides complicated/complex/unclear position',
 'Position.on.Business.Regulations_Candidate provides complicated/complex/unclear position',
 'Position.on.Business.Regulations_Candidat

In [56]:
fwd_sel_df = reduced_df[['General Status']+best_X_lst]

Now we want to call the aic function:


In [57]:
aic(fwd_sel_df[best_X_lst].astype(float),fwd_sel_df['General Status'])

Optimization terminated successfully.
         Current function value: 0.602383
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.601459
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.602383
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.595734
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.595273
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.592111
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.591846
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.587725
         Iterations 5
Optimization terminated successfully.
         Current function value: 0.554955
         Iterations 6
Optimization terminated successfully.
         Current function value: 0.532697
  

LinAlgError: Singular matrix